# Funciones

## Desarrollo de funciones


Para este notebook se desarrollarán las funciones que se van a utilizar en el main.py que será utilizado para hacer el deploy de nuestra API. Tenemos 5 funciones que desarrollar, las cuales son las siguientes:

- **def PlayTimeGenre( genero : str )**: Debe devolver año con mas horas jugadas para dicho género.

- **def UserForGenre( genero : str )**: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

- **def UsersRecommend( año : int )**: Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

- **def UsersNotRecommend( año : int )**: Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

- **def sentiment_analysis( año : int )**: Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Para las últimas tres funciones tenemos que hacer utilización de la siguiente instrucción:

"En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1."

No se agregó esta parte en la ETL ya que ya se empieza a trabajar con funciones, es por eso que se agrego en este notebook, sin embargo, esa función no se encontrara en el main.py final. Además gracias a este analisis es necesario modificar uno de nuestros archivos parquet, por lo que se creará uno nuevo y es el que se utilizará al final en el proyecto.


### Funcion SentimenAnalysis:
Se utilizará la columna de reviews y NLP para evaluar si es la reseña es 'negativa', 'neutral' o 'positiva'. Estos resultados reemplazarán las reseñas y serán clasificadas como 0, 1 y 2.

In [786]:
#Importamos las librerias a utilizar en el archivo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [787]:
#Importamos nuestros datos del review a un dataframe
df_revs = pd.read_parquet(r'..\DataParquet\\AustReviews.parquet')

In [788]:
df_revs.head()

,user_id,item_id,helpful,recommend,review
21123,--000--,1250,1 of 1 people (100%) found this review helpful,True,‡∏´‡∏ô‡∏∏‡∏Å‡∏Ñ‡∏±‡∏ö ‡πÅ‡∏ô‡∏∞‡∏ô‡∏≥ 10/10
58209,--ace--,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :) ...
58208,--ace--,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed
32142,--ionex--,730,No ratings yet,True,"it done brokeded on me, the game no longer wor..."
32143,--ionex--,105600,No ratings yet,True,"It's an amazing game, and im glad that mac use..."


Comenzaremos a hacer una limpieza del texto, buscar caracteres que puedan romper el código y ordenar todas las reseñas.

In [789]:
df_revs.sort_values(by= 'review', ascending= True)

,user_id,item_id,helpful,recommend,review
40959,stionk,570,No ratings yet,True,
35990,76561198125368693,24010,0 of 1 people (0%) found this review helpful,True,
25295,76561198098017317,730,No ratings yet,True,
45215,CBMC,289650,6 of 10 people (60%) found this review helpful,False,
37128,AlienMan180,8930,No ratings yet,True,
...,...,...,...,...,...
45777,komarexpl,233450,29 of 32 people (91%) found this review helpful,True,"≈öwietna gra! Mimo, i≈º jest ciƒÖgle w fazie a..."
2087,LapFucksTrax,211820,No ratings yet,True,ùñ≥ùñæùóãùóãùñ∫ùóãùóÇùñ∫ ùñªùóéùóç ...
18562,wowsteamid,383080,3 of 5 people (60%) found this review helpful,True,üëåüëÄüëåüëÄüëåüëÄüëåüëÄüëåüëÄ good ...
29932,FictionDaddy,292030,No ratings yet,True,üëåüëåüëåüëåüëå


Podemos observar que al principio tenemos reseñas vacias y despues al final tenemos reseñas con caracteres aleatorio o que no nos van a servir para el analisis, esas reseñas no serán eliminadas, si no que serán consideradas como neutrales (se les asignara el valor 1)

In [790]:
#Todos los comentarios a minusculas y llenamos las reseñas vacias con 'neutral review'
df_test = df_revs
revs = list(df_test['review'])

for i in range(len(revs)):
    revs[i] = revs[i].lstrip()
    if revs[i] == '' or revs[i] == ' ' or revs[i] == '  ':
        revs[i] = 'neutral review'
    revs[i] = revs[i].lower()


In [791]:
characters = '‡∏™‡∏ô‡∏∏‡∏å‡∏°‡∏≤‡∏åô•øæäâç°ïûºæ∏î•¥ïûºõú0ñ‰∏∫‰∫ü‰∏çêéâôºåáé©âà∞âø√≈öù§~{íµ≥{|'
char_list = []
for i in range(len(characters)):
    char_list.append(characters[i])
char_list.append('!!!!!')
char_list.append('//')

In [792]:
#Con esto limpiamos nuestras reviews, 
for i in range(len(revs)):
    for j in char_list:
        if j in revs[i]:
            revs[i] = 'neutral review'
            break

        if len(revs[i]) < 20:
            revs[i] = 'neutral review'
            break

In [793]:
df_test['review'] = revs

df_test.sort_values(by= 'review', ascending= True)

,user_id,item_id,helpful,recommend,review
29684,EyesScream,730,1 of 1 people (100%) found this review helpful,True,"""... i've spent hundreds of dollars on slightl..."
41027,76561198093360882,248820,No ratings yet,True,"""5 stars! a+! 4 more stars!"" is wah i wish i c..."
53364,76561198075417014,243870,No ratings yet,True,"""a cautious recommendation. an enjoyable 3rd p..."
16861,76561198107093846,440,1 of 1 people (100%) found this review helpful,True,"""a classic with loads of fun to be had. pick b..."
51752,isaacredfist,1840,1 of 1 people (100%) found this review helpful,True,"""all though i havent used it on my computer it..."
...,...,...,...,...,...
25960,76561198063321128,550,No ratings yet,True,zombies with mods i love this game
39709,76561198158100654,35420,No ratings yet,False,zombies zijn ontzichtbaar dus er is niet veel aan
26478,BuckysButthol,550,No ratings yet,True,"zombies, blood & humor. love zombie apocalypse..."
46170,assassin_bunny,273110,1 of 1 people (100%) found this review helpful,True,"zombies.everyone loves zombies, which is why t..."


Como se puede observar ya tenemos las reseñas filtradas, para el filtrado se neutralizaron las reseñas que contenían simbolos muy raros o que no iban a aportar nada a la reseña. Despues se neutralizaron todas las reseñas que tenían una longitud de caracteres menor a 15, ya que si la reseña es más pequeña a eso no va a aportar nada con tan pocos caracteres.

Ahora para empezar a entrenar nuestro sistema tenemos que tener un tren de pruebas con unos datos ya clasificados, vamos a utilizar ChatGPT para generar 100 reseñeas buenas, 100 malas y 100 neutrales. Todo esto lo vamos a guardar en el archivo Chat_revs.csv, para poder entrenar nuestro sistema.

In [794]:
#Leemos el archivo, limpiamos valores vacios y lo mostramos
df_test_train = pd.read_csv('Chat_Sents.csv', encoding='latin1')
df_test_train['review'].fillna('NaN', inplace=True)
df_test_train['Sentiment'].fillna(1, inplace=True)
df_test_train.head()

,ID,review,Sentiment
0,1.0,"Stunning graphics, immersive gameplay.",2.0
1,2.0,"Compelling story, highly recommended.",2.0
2,3.0,"Intuitive controls, enjoyable experience.",2.0
3,4.0,"Virtually bug-free, incredibly polished.",2.0
4,5.0,Engaging from start to finish.,2.0


In [795]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_test_train['review'], df_test_train['Sentiment'], test_size=0.2, random_state=42)

# Extracción de características usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000) # Puedes ajustar el número máximo de características según tus necesidades
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entrenamiento del modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)

# Predicción y evaluación del modelo
y_pred = svm_model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.79      0.86        19
         1.0       0.86      0.86      0.86        21
         2.0       0.75      0.86      0.80        21

    accuracy                           0.84        61
   macro avg       0.85      0.83      0.84        61
weighted avg       0.85      0.84      0.84        61



Podemos observar que con estas configuraciones nuestro sistema logra una presición bastante buena, por lo que lo vamos a dejar con estos parametros y ahora vamos a someter nuestro dataset del proyecto al sistema para poder clasificar nuestras reseñas y ver los resultados con una pequeña muestra.

Creamos una función a la que le pasaremos individualmente nuestras reseñas para poder ser analizadas y clasificadas. La función nos va a regresar los valores del 0 al 2.

In [796]:
def sentimiento(x):
    # Extracción de características utilizando el vectorizador TF-IDF
    nueva_resena_tfidf = tfidf_vectorizer.transform([x])

    # Predicción utilizando el modelo SVM entrenado
    prediccion = svm_model.predict(nueva_resena_tfidf)

    # Retornamos la predicción
    return prediccion

In [797]:
#vemos nuestros datos antes del procesamiento
resenas = df_test['review']
resenas

21123                                       neutral review
58209    one of the funnyest games that is animated :) ...
58208                        the best game i ever plllayed
32142    it done brokeded on me, the game no longer wor...
32143    it's an amazing game, and im glad that mac use...
                               ...                        
55932    fantastic game! lots of gamemodes and large va...
55931                                       neutral review
55939    a gem.it's level of stupidity is just overwhem...
55936              dis game is action packed and thrilling
42588    ok, ok, i am a vietnamese and excuse me for th...
Name: review, Length: 59305, dtype: object

In [798]:
#Ingresamos nuestras reseñas analizadas
Sentiment_Analysis = []

for i in resenas:
    Sentiment_Analysis.append(sentimiento(i))

#Hacemos un acomodo para extraer los datos
sentimientos = []
for i in range(len(Sentiment_Analysis)):
    sentimientos.append(int(Sentiment_Analysis[i][0]))

In [799]:
#Importamos nuestros datos del review a un dataframe
df_revs = pd.read_parquet(r'..\DataParquet\\AustReviews.parquet')

In [800]:
#Generamos nuestro dataframe final para las reviews
df_reviews_clean = pd.DataFrame()

df_reviews_clean['User_Id'] = df_revs['user_id']
df_reviews_clean['Item_Id'] = df_revs['item_id']
df_reviews_clean['Review'] = df_revs['review']
df_reviews_clean['Sentiment_Analysis'] =  sentimientos

In [801]:
df_reviews_clean.head()

,User_Id,Item_Id,Review,Sentiment_Analysis
21123,--000--,1250,‡∏´‡∏ô‡∏∏‡∏Å‡∏Ñ‡∏±‡∏ö ‡πÅ‡∏ô‡∏∞‡∏ô‡∏≥ 10/10,1
58209,--ace--,113200,One Of The Funnyest Games That Is Animated :) ...,1
58208,--ace--,440,the best game i ever plllayed,2
32142,--ionex--,730,"it done brokeded on me, the game no longer wor...",2
32143,--ionex--,105600,"It's an amazing game, and im glad that mac use...",2


Podemos observar que se logró una clasificación de nuestras reseñas, por lo tanto, vamos a volver a hacer este dataframe pero sin la columna de 'Review' ya que solo utilizaremos los valores del analisis de sentimiento, por último vamos a exportar estos reaultados a un parquet diferente  para tener nuestros datos ya limpios al 100%.

Nuestros dataframe final quedará de la siguiente manera:

In [802]:
#Generamos nuestro dataframe final para las reviews
df_reviews_clean = pd.DataFrame()

df_reviews_clean['User_Id'] = df_revs['user_id']
df_reviews_clean['Item_Id'] = df_revs['item_id']
df_reviews_clean['sentiment_analysis'] =  sentimientos

In [803]:
df_reviews_clean.head()

,User_Id,Item_Id,sentiment_analysis
21123,--000--,1250,1
58209,--ace--,113200,1
58208,--ace--,440,2
32142,--ionex--,730,2
32143,--ionex--,105600,2


In [804]:
#Exportamos nuestros datos a un archivos parquet.
df_reviews_clean.to_parquet(r'..\DataParquet\\CleanReviews.parquet')

In [805]:
#Solo para comprobar leemos nuestro parquet nuevo para ver que todo salió ok
df_reviews_final = pd.read_parquet(r'..\DataParquet\\CleanReviews.parquet')
df_reviews_final.head()


,User_Id,Item_Id,sentiment_analysis
21123,--000--,1250,1
58209,--ace--,113200,1
58208,--ace--,440,2
32142,--ionex--,730,2
32143,--ionex--,105600,2


Con esto terminamos la parte del sistema de ML para clasificar las resñeas en buenas, neutras o malas. Ahora procedemos a desarrollar las demás fucniones.

## Función 01: 

### def PlayTimeGenre( genero : str ): 
Debe devolver año con mas horas jugadas para dicho género.
 
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [806]:
def load_data():
    df_steam_games = pd.read_parquet(r'..\DataParquet\\SteamGames.parquet')
    df_australian_items_ids = pd.read_parquet(r'..\DataParquet\\AustItems.parquet')
    df_australian_items_playtime = pd.read_parquet(r'..\DataParquet\\AustItemsExpand.parquet')
    df_reviews = pd.read_parquet(r'..\DataParquet\\CleanReviews.parquet')
    return df_steam_games,df_australian_items_ids,df_australian_items_playtime,df_reviews

In [807]:
def PlayTimeGenre(genero):
    df_steam_games,df_australian_items_ids,df_australian_items_playtime,df_reviews = load_data()

    #Se nos va a dar una variables 'genero' a buscar
    tiene_genero = []

    for i in df_steam_games['genres']:
        for j in i:
            if genero.upper() == j.upper():
                tiene_genero.append('YES')
                break
            else:
                tiene_genero.append('NO')
                break

    df_steam_games['tiene_genero'] = tiene_genero
    #Hacemos un df nuevo de las filas que contienen el genero para obtener los IDS
    df_datos_con_genero = df_steam_games[df_steam_games['tiene_genero'].isin(['YES'])]
    df_datos_con_genero['id'] = df_datos_con_genero['id'].astype(int)
    Ids_con_genero = df_datos_con_genero['id'].astype(int).astype(str)

    #Encontramos las horas para los ids
    df_horas_id = df_australian_items_playtime[df_australian_items_playtime['item_id'].isin(Ids_con_genero)]

    #Creamos un dataframe con los ids y la sumas de sus horas
    df_suma_por_id = df_horas_id.groupby('item_id')['playtime_2weeks'].sum().reset_index()

    #Extramemos la fila que contenga el valor de horas mayor para obtener su ID
    indice_maximo = df_suma_por_id['playtime_2weeks'].idxmax()

    # Extraer la fila con el valor más alto
    fila_maximo = df_suma_por_id.loc[indice_maximo]
    IDfinal = fila_maximo['item_id']

    #Ya tenemos el Id con el mayor tiempo de horas jugadas para el genero propuesto, ahora es encontrar el año del juego que tiene ese id
    dato_final = df_datos_con_genero[df_datos_con_genero['id'].isin([int(IDfinal)])]

    #El resultado del año es:
    anio = dato_final['year']

    return print(f'El año con más horas jugadas para el genero {genero.upper()}, es el año: {anio.values[0]}.')



In [808]:
#Comprobación de funcionamiento:
TestFun01 = PlayTimeGenre('rpg')
print(TestFun01)

C:\Users\jared\AppData\Local\Temp\ipykernel_11012\667318879.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_datos_con_genero['id'] = df_datos_con_genero['id'].astype(int)


El año con más horas jugadas para el genero RPG, es el año: 2015.
None


## Función 02:

### def UserForGenre( genero : str ): 

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [809]:
def UserForGenre(genero):
    df_steam_games,df_australian_items_ids,df_australian_items_playtime,df_reviews = load_data()

    #Extraemos los datos de los generos coincidentes como en la primera función

    #Se nos va a dar una variables 'genero' a buscar
    tiene_genero = []

    for i in df_steam_games['genres']:
        for j in i:
            if genero.upper() == j.upper():
                tiene_genero.append('YES')
                break
            else:
                tiene_genero.append('NO')
                break

    df_steam_games['tiene_genero'] = tiene_genero

    #Apartamos nuestros datos del genero pedido
    df_datos_genero = df_steam_games[df_steam_games['tiene_genero'].isin(['YES'])]
    df_datos_genero['id'] = df_datos_genero['id'].astype(int).astype(str)

    #Ahora extraemos los id para poder encontrar el ID del usuario
    id_juegos = df_datos_genero['id']

    #Ahora separamos los datos de nuestros ids del dataframe playtime
    df_userIds_playtime = df_australian_items_playtime[df_australian_items_playtime['item_id'].isin(list(id_juegos))]

    #Obtenemos el jugador con más horas jugadas para el genero
    df_suma_por_id = df_userIds_playtime.groupby('user_id')['playtime_2weeks'].sum().reset_index()

    #Extramemos la fila que contenga el valor de horas mayor para obtener el id del user
    indice_maximo = df_suma_por_id['playtime_2weeks'].max()
    datos = df_suma_por_id[df_suma_por_id['playtime_2weeks'].isin([indice_maximo])]
    user = datos['user_id'].values[0]

    #Ya tenemos el usuario, así que ahora extraemos los juegos que ha jugado
    df_user_games_played = df_australian_items_playtime[df_australian_items_playtime['user_id'].isin([user])]
    juegos = df_user_games_played['item_id']

    #Ya tenemos los juegos, ahora hay que filtrarlos del df de genero y extraer los años que jugó
    df_user_games_played_per_year = df_datos_genero[df_datos_genero['id'].isin(juegos)]
    df_user_games_played_per_year = df_user_games_played_per_year.sort_values(by='year', ascending= True)
    años = df_user_games_played_per_year['year'].unique()

    #Ya tenemos los años, ahora buscamos los juegos por año en el df de generos, luegos esos ids hacemos las sumas de tiempo jugado y lo agregamos a una lista
    final = []
    for i in años:
        df_año_genero = df_datos_genero[df_datos_genero['year'].isin([i])]
        df_juego_x_año = df_user_games_played[df_user_games_played['item_id'].isin(df_año_genero['id'])]
        suma_horas = df_juego_x_año['playtime_2weeks'].sum()

        final.append(f'Año: {i}, Horas: {suma_horas/60}')

    return print(f'Usuario con más horas jugadas para el genero {genero}: {user}\n{final}')


In [810]:
#Comprobación de funcionamiento:
usuario = UserForGenre('Action')
print(usuario)

C:\Users\jared\AppData\Local\Temp\ipykernel_11012\2656226853.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_datos_genero['id'] = df_datos_genero['id'].astype(int).astype(str)


Usuario con más horas jugadas para el genero Action: 76561198106488845
['Año: 2004, Horas: 5.9', 'Año: 2006, Horas: 0.0', 'Año: 2010, Horas: 79.06666666666666', 'Año: 2012, Horas: 247.55', 'Año: 2013, Horas: 542.6', 'Año: 2014, Horas: 77.36666666666666', 'Año: 2015, Horas: 571.2833333333333', 'Año: 2016, Horas: 173.7', 'Año: 2017, Horas: 208.31666666666666', 'Año: Jun 2015, Horas: 0.0']
None


## Función 03:

### def UsersRecommend( año : int ):

Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [811]:
def UsersRecommend(anio):
    df_steam_games,df_australian_items_ids,df_australian_items_playtime,df_reviews = load_data()

    anio = str(anio)

    #Extraemos los datos de los juegos para el año dado

    df_juego_año = df_steam_games[df_steam_games['year'].isin([anio])]
    df_juego_año['id'] = df_juego_año['id'].astype(int).astype(str)

    #Extraemos los IDs de los juegos
    Ids = df_juego_año['id']

    #Extraemos los datos de los juegos de ese año
    df_recomendacion_juegos_año = df_reviews[df_reviews['Item_Id'].isin(Ids)]

    #Extraemos los juegos más recomendado (calificación de 2)
    df_juegos_recomendados = df_recomendacion_juegos_año[df_recomendacion_juegos_año['sentiment_analysis'].isin([2 and 1])]

    #Obtenemos el jugador con más horas jugadas para el genero
    df_suma_por_id = df_juegos_recomendados.groupby('Item_Id')['sentiment_analysis'].sum().reset_index()
    df_suma_por_id = df_suma_por_id.sort_values(by= 'sentiment_analysis', ascending= False)

    #Extraemos el juego y las horas:
    id_game = []

    for i in df_suma_por_id[0:3].values:
        for j in i:
            if type(j) == int:
                continue
            else:
                id_game.append(j)

    #Ahora extraemos el nombre del juego para entregar la respuesta final
    titulos = []
    cont = 0
    for i in id_game:
        cont+= 1
        name = df_juego_año[df_juego_año['id'] == i]
        titulos.append(f'Puesto {cont}: {name.iloc[0]['title']}')
    
    return titulos


In [812]:
#Comprobación de funcionamiento:
Recomendacion = UsersRecommend(2005)
print(Recomendacion)

['Puesto 1: Star Wars: Battlefront 2 (Classic, 2005)', 'Puesto 2: Grand Theft Auto: San Andreas', 'Puesto 3: STAR WARS™ Republic Commando™']


C:\Users\jared\AppData\Local\Temp\ipykernel_11012\780627508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_juego_año['id'] = df_juego_año['id'].astype(int).astype(str)


## Función 04:

### def UsersNotRecommend( año : int ):

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [813]:
def UsersNotRecommend(anio):
    df_steam_games,df_australian_items_ids,df_australian_items_playtime,df_reviews = load_data()
    
    anio = str(anio)

    #Extraemos los datos de los juegos para el año dado

    df_juego_año = df_steam_games[df_steam_games['year'].isin([anio])]
    df_juego_año['id'] = df_juego_año['id'].astype(int).astype(str)

    #Extraemos los IDs de los juegos
    Ids = df_juego_año['id']

    #Extraemos los datos de los juegos de ese año
    df_recomendacion_juegos_año = df_reviews[df_reviews['Item_Id'].isin(Ids)]

    #Extraemos los juegos más recomendado (calificación de 2)
    df_juegos_recomendados = df_recomendacion_juegos_año[df_recomendacion_juegos_año['sentiment_analysis'].isin([0])]

    #Obtenemos el item id con un groupby
    df_suma_por_id = df_juegos_recomendados.groupby('Item_Id')['sentiment_analysis'].count().reset_index()
    df_suma_por_id = df_suma_por_id.sort_values(by= 'sentiment_analysis', ascending= False)

    #Extraemos el juego y las horas:
    id_game = []

    for i in df_suma_por_id[0:3].values:
        for j in i:
            if type(j) == int:
                continue
            else:
                id_game.append(j)

    #Ahora extraemos el nombre del juego para entregar la respuesta final
    titulos = []
    cont = 0
    for i in id_game:
        cont+= 1
        name = df_juego_año[df_juego_año['id'] == i]
        titulos.append(f'Puesto {cont}: {name.iloc[0]['title']}')

    return titulos

In [814]:
#Comprobación de funcionamiento:
Recomendacion = UsersRecommend(1998)
print(Recomendacion)

['Puesto 1: Half-Life', 'Puesto 2: Fallout 2: A Post Nuclear Role Playing Game', "Puesto 3: Oddworld: Abe's Exoddus®"]


C:\Users\jared\AppData\Local\Temp\ipykernel_11012\780627508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_juego_año['id'] = df_juego_año['id'].astype(int).astype(str)


## Función 05:

## def sentiment_analysis( año : int ): 

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [815]:
def sentiment_analysis(anio):
    df_steam_games,df_australian_items_ids,df_australian_items_playtime,df_reviews = load_data()

    anio = str(anio)

    #Extraemos los ids de los juegos de ese año
    df_ids_año = df_steam_games[df_steam_games['year'].isin([anio])]
    ids_juegos = df_ids_año['id'].astype(int).astype(str)

    #Extraemos los sentiments analysis para esos ids
    df_ids_sentiment = df_reviews[df_reviews['Item_Id'].isin(ids_juegos)]

    #Creamos una nueva columna para clasificar si es buena, neutral o mala
    clasificacion = []
    for i in df_ids_sentiment['sentiment_analysis']:
        if i == 0:
            clasificacion.append('Negative')
        if i == 1:
            clasificacion.append('Neutral')
        if i == 2:
            clasificacion.append('Positive')

    df_ids_sentiment['Clasification'] = clasificacion
        
    #Creamos un DF para tener solo la columna de clasificación y sentimient
    datos = pd.DataFrame()
    datos['Clasificacion'] = df_ids_sentiment['Clasification']
    datos['sentiment_analysis'] = df_ids_sentiment['sentiment_analysis']

    #Ahora ordenamos
    df_suma_clasificacion = datos.groupby('Clasificacion')['sentiment_analysis'].count().reset_index()
    df_suma_clasificacion = df_suma_clasificacion.sort_values(by= 'sentiment_analysis', ascending= False)

    #Ordenamos nuestros datos para dar la respuesta final
    respuesta = []
    for i in df_suma_clasificacion.values:
        for j in i:
            respuesta.append(f'{i[0]} = {i[1]}')
            break
    
    return respuesta

In [816]:
#Comprobación de funcionamiento:
sentimientos = sentiment_analysis(2005)
print(sentimientos)

['Neutral = 165', 'Positive = 56', 'Negative = 27']


C:\Users\jared\AppData\Local\Temp\ipykernel_11012\2324470447.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ids_sentiment['Clasification'] = clasificacion


Con esto terminamos las 5 funciones propuestas para el proyecto. Ahora toca hacer el deploy en FastApi de nuestro proyecto, por lo cual se va genear un archivo main.py con nuestras funciones un poco modificadas para que puedan ser usadas por FastApi.